# LSTM with Attention Keras
Rob Trying to make a LSTM with attention

In [102]:
import tensorflow
import keras
import numpy as np
import pandas as pd

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

from nltk.tokenize import sent_tokenize, word_tokenize


import re

## Data Preprocessing

In [3]:
## Load comments by game
files = [
    'Bears_vs_Packers__2017-09-28_comment_sentiment.pickle',
    'Broncos_vs_Chiefs__2017-10-30_comment_sentiment.pickle',
    'Chargers_vs_Cowboys__2017-11-23_comment_sentiment.pickle',
    'Chiefs_vs_Patriots__2017-09-07_comment_sentiment.pickle',
    'Chiefs_vs_Raiders__2017-10-19_comment_sentiment.pickle',
    'Cowboys_vs_Cardinals__2017-09-25_comment_sentiment.pickle',
    'Cowboys_vs_Raiders__2017-12-17_comment_sentiment.pickle',
    'Eagles_vs_Panthers__2017-10-12_comment_sentiment.pickle',
    'Falcons_vs_Buccaneers__2017-12-18_comment_sentiment.pickle',
    'Falcons_vs_Patriots__2017-10-22_comment_sentiment.pickle',
    'Falcons_vs_Seahawks__2017-11-20_comment_sentiment.pickle',
    'Giants_vs_Cowboys__2017-09-10_comment_sentiment.pickle',
    'Jaguars_vs_Patriots__2018-01-21_comment_sentiment.pickle',
    'Lions_vs_Giants__2017-09-18_comment_sentiment.pickle',
    'Lions_vs_Packers__2017-11-06_comment_sentiment.pickle',
    'Packers_vs_Panthers__2017-12-17_comment_sentiment.pickle',
    'Packers_vs_Vikings__2017-10-15_comment_sentiment.pickle',
    'Patriots_vs_Dolphins__2017-12-11_comment_sentiment.pickle',
    'Raiders_vs_Eagles__2017-12-25_comment_sentiment.pickle',
    'Raiders_vs_Redskins__2017-09-24_comment_sentiment.pickle',
    'Rams_vs_49ers__2017-09-21_comment_sentiment.pickle',
    'Redskins_vs_Chiefs__2017-10-02_comment_sentiment.pickle',
    'Redskins_vs_Cowboys__2017-11-30_comment_sentiment.pickle',
    'Redskins_vs_Eagles__2017-10-23_comment_sentiment.pickle',
    'Saints_vs_Falcons__2017-12-07_comment_sentiment.pickle',
    'Saints_vs_Vikings__2017-09-11_comment_sentiment.pickle',
    'Seahawks_vs_Cardinals__2017-11-09_comment_sentiment.pickle',
    'Steelers_vs_Bengals__2017-12-04_comment_sentiment.pickle',
    'Steelers_vs_Lions__2017-10-29_comment_sentiment.pickle',
    'Texans_vs_Bengals__2017-09-14_comment_sentiment.pickle',
    'Vikings_vs_Packers__2017-12-23_comment_sentiment.pickle',
    'Vikings_vs_Panthers__2017-12-10_comment_sentiment.pickle']

path = "/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/"

for index, filename in enumerate(files):
    print(path+filename)
    if index == 0:
        data = pd.read_pickle(path+filename)
        # print(data.head())
    else:
        temp_data = pd.read_pickle(path+filename)
        data = data.append(temp_data)
        # print(data.head())

/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Bears_vs_Packers__2017-09-28_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Broncos_vs_Chiefs__2017-10-30_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Chargers_vs_Cowboys__2017-11-23_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Chiefs_vs_Patriots__2017-09-07_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Chiefs_vs_Raiders__2017-10-19_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Cowboys_vs_Cardinals__2017-09-25_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Game_Data/Cowboys_vs_Raiders__2017-12-17_comment_sentiment.pickle
/home/robmulla/Documents/W266/w266_final_project_merged/data/Clean_Gam

In [4]:
data[data.author == 'Scaryclouds'].comment_body.head()

8784                                Fisher you fat fuck! 
8814    Hmm, looks like a bad spot, I think Smith got ...
8825                           Yea, first down for sure. 
8843    God damn, our o-line is just terrible at run b...
8860      Part of it is injuries to our interior o-line. 
Name: comment_body, dtype: object

In [9]:
data.shape

(588378, 43)

In [12]:
data_fans = data.loc[(data['author_flair'] == data['home_team']) | (data['author_flair'] == data['away_team'])]
data_fans.shape

(211168, 43)

In [29]:
def gametype(row):
    if (row > 0.9) | (row < 0.1):
        return 'Blowout'
    elif (row > 0.4) & (row < 0.6):
        return 'Close'
    else:
        return 'Middleground'

data['game_suspense'] = data['homeWinPercentage'].apply(gametype)
print(data['game_suspense'.)
print(data_close.shape)

(225254, 43)
(153364, 43)


In [40]:
data.groupby('game_suspense').count()['author']

game_suspense
Blowout         170925
Close           114181
Middleground    303272
Name: author, dtype: int64

In [94]:
data_close_blowout = data.loc[data['game_suspense'] != 'Middleground'].copy()

data_close_blowout['game_suspense_binary'] = data_close_blowout['game_suspense'].apply(lambda x: 1 if x =='Blowout' else 0)

labels = data_close_blowout['game_suspense_binary'].values


In [104]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

docs = data_close_blowout['comment_body']
# Lowercase words
X_data = docs.map(lambda x: x.lower())

# Convert links to word 'postedhyperlinkvalue'
X_data = X_data.map(lambda x: re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?\S", \
                                     "postedhyperlinkvalue", x))

# Convert digits to 'd'
X_data = X_data.map(lambda x: re.sub('\d', 'd', x))

# Tokenize words
X_data = X_data.map(lambda x: word_tokenize(x))

# Remove stop words
stop_words = set(stopwords.words('english'))
X_data = X_data.map(lambda x: [w for w in x if not w in stop_words])

# Word stemming

ps = PorterStemmer()
X_data = X_data.map(lambda x: [ps.stem(w) for w in x])

In [106]:
vocab_size = 100000
encoded_docs = [one_hot(str(d), vocab_size) for d in X_data]

# pad documents to a max length of 500 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

## Create Model

In [108]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=3, verbose=1, validation_split=0.33)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 8)            800000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               43600     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 843,701
Trainable params: 843,701
Non-trainable params: 0
_________________________________________________________________
None
Train on 191021 samples, validate on 94085 samples
Epoch 1/3
191021/191021 [==============================] - 260s 1ms/step - loss: 0.6523 - acc: 0.6428 - val_loss: 0.7265 - val_acc: 0.5116
Epoch 2/3
191021/191021 [==============================] - 260s 1ms/step - loss: 0.6520 - acc: 0.6428 - val_loss: 0.7229 - val_acc: 0.5116
Epoch 3/3
285106/285106 [==============================] - 71s 251

## Don't Remove Stop Words

In [109]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

docs = data_close_blowout['comment_body']
# Lowercase words
X_data = docs.map(lambda x: x.lower())

# Convert links to word 'postedhyperlinkvalue'
X_data = X_data.map(lambda x: re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?\S", \
                                     "postedhyperlinkvalue", x))

# Convert digits to 'd'
X_data = X_data.map(lambda x: re.sub('\d', 'd', x))

# Tokenize words
X_data = X_data.map(lambda x: word_tokenize(x))

In [110]:
vocab_size = 100000
encoded_docs = [one_hot(str(d), vocab_size) for d in X_data]

# pad documents to a max length of 500 words
max_length = 100
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=3, verbose=1, validation_split=0.33)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 8)            800000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               43600     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 843,701
Trainable params: 843,701
Non-trainable params: 0
_________________________________________________________________
None
Train on 191021 samples, validate on 94085 samples
Epoch 1/3
191021/191021 [==============================] - 289s 2ms/step - loss: 0.6523 - acc: 0.6428 - val_loss: 0.7228 - val_acc: 0.5116
Epoch 2/3
191021/191021 [==============================] - 289s 2ms/step - loss: 0.6521 - acc: 0.6428 - val_loss: 0.7237 - val_acc: 0.5116
Epoch 3/3
285106/285106 [==============================] - 79s 277